# Quality controls

In [ ]:
import os
import json
import sys

In [ ]:
sys.path.append(isc_path)
import scanpy as sc
import pandas as pd
import numpy as np
import anndata as ad
from scmIonSuppressionCorrection.src import const
from scmIonSuppressionCorrection.src.evaluation import compare_pre_post_correction, plot_deviations_between_adatas

%matplotlib inline
# Default parameters

In [ ]:
deconvolution_params = {}
am_adatas = {}
corr_am_adatas = {}

for sample in config['runtime']['samples']:
    
    config_path = os.path.join(config['runtime']['spacem_dataset_path'], 
                               sample, 
                               config['input']['spacem_config_file'])
    
    if os.path.exists(config_path):
        with open(config_path) as json_file:
            data = json.load(json_file)
        deconvolution_params[sample] = data['single_cell_analysis']

    sample_path = os.path.join(config['runtime']['out_folder'], 
                               sample, 
                               config['output']['write_sample_folder_path'])
    
    am_adatas[sample] = sc.read(os.path.join(config['runtime']['spacem_dataset_path'],
                               sample,
                               config['input']['am_adata_file']))
    corr_am_adatas[sample] = sc.read(os.path.join(config['runtime']['out_folder'], 
                               sample, 
                               config['output']['write_sample_folder_path'], 
                               config['output']['file_names']['corrected_am_adata']))


## Correction slopes

In [ ]:
# reshape data for immediate analysis
am_adata = ad.concat(am_adatas, label='well', index_unique="_", merge="first")
corr_am_adata = ad.concat(corr_am_adatas, label='well', index_unique="_", merge="first")

In [ ]:
am_adata.obs[const.TPO] = corr_am_adata.obs[const.TPO]
am_adata.var[const.FPAI] = corr_am_adata.var[const.FPAI]

compare_pre_post_correction(adata = am_adata, 
                            adata_cor = corr_am_adata, 
                            proportion_threshold = config['correction']['correction_proportion_threshold'], 
                            normalized = True)

## Deconvolution

The following table shows the individual deconvolution parameters of the different wells, as documented in the respective config.json files. These parameters were used if the flag "use_data_from_spacem_configuration" is True. Otherwise, the parameters from the configuration above were used.

In [ ]:
deconv_table = pd.DataFrame(deconvolution_params).T
deconv_table

In [ ]:
adata = ad.read(os.path.join(config['runtime']['out_folder'], config['output']['file_names']['adata']))
gen_adata = ad.read(os.path.join(config['runtime']['out_folder'], config['output']['file_names']['generated_adata']))
adata_cor = ad.read(os.path.join(config['runtime']['out_folder'], config['output']['file_names']['corrected_adata']))

In [ ]:
plot_deviations_between_adatas(adata, gen_adata, adata_cor, well_name=const.SAMPLE_COL)